In [6]:
!pip install transformers-interpret shap

In [7]:
!pip uninstall shap pandas -y
!pip install shap pandas

Found existing installation: shap 0.48.0
Uninstalling shap-0.48.0:
  Successfully uninstalled shap-0.48.0
Found existing installation: pandas 2.3.1
Uninstalling pandas-2.3.1:
  Successfully uninstalled pandas-2.3.1
  Using cached shap-0.48.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (25 kB)
  Using cached pandas-2.3.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (91 kB)
Using cached shap-0.48.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
Using cached pandas-2.3.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.4 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.1 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.1 which is incompatible.
cudf-cu12 25.6.0 re

In [8]:
import shap
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers_interpret import SequenceClassificationExplainer

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
# Cargar modelo ya entrenado
model_name = "dccuchile/bert-base-spanish-wwm-cased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Crear el explainer
explainer = SequenceClassificationExplainer(
    model=model,
    tokenizer=tokenizer
)

# Explicar una predicción concreta
texto = "Paciente con síntomas psicóticos agudos, delira y presenta alucinaciones."
word_attributions = explainer(texto)

# Mostrar las palabras más relevantes visualmente
explainer.visualize()


In [ ]:
# Cargar modelo/tokenizer y poner en modo eval
model.eval()
def predict(texts):
    tokens = tokenizer(texts, padding=True, truncation=True, return_tensors='pt').to(model.device)
    outputs = model(**tokens)
    return torch.nn.functional.softmax(outputs.logits, dim=1).detach().cpu().numpy()

# Usar el explainer de SHAP
explainer = shap.Explainer(predict, tokenizer)
shap_values = explainer(["Paciente con delirio paranoide y pérdida de contacto con la realidad."])

# Visualizar
shap.plots.text(shap_values[0])
